# Step 3: Define the Data Model

### The data model is organized as a star schema
- The goal is to **trade storage space and consistency checks for speed**.
- The key is for queries to perform **as few joins as possible**, which are an expensive but unavoidable operation.
- For this reason, dimension schemas will contain all columns relevant to the dimension at hand, even if this **information is repeated** in other dimensions, rather than normalizing.

### Design principles that I followed
- *Pure star schema*, as opposed to a snowflake schema. There will be a lot of repition, for the reason stated above, given that, as it is, storage is not an issue.
- I will use a dimension's natural key, if any, as its *primary key* if the key is derived from a single data source. This principle also applies to compound primariy keys (for instance, in the ```route_dim``` table).
- All dimension columns whose values are codes or abbreviations will have a corresponding *descriptive column* (with the ```_desc``` suffix).
- *No measures in dimension tables*. So, I used demographics and temperature tables only to derive static population, ethnical and weather categories.

## Model description

### Abstract data model

I will start by presenting what I consider as the core entities that are relevant to my understanding of this model's domain:

![Conceptual Data Model](images/de-capstone-abstract-data-model.png)

**NOTE**: I preferred to draw a more abstract and simpler (fewer symbols) conceptual data model diagram instead of the more common E-R diagram, because in my opinion the latter includes detail that in my opinion is unnecessary when one is already set on a star model. The intention is to highlight the main atoms and how they they are related. The multiplicities represent business rules and will be useful later on when designing data validation.

### Core entities description

I will describe only those entities whose precise meaning might not be obvious.

#### Route
A particular airline and flight number combination that flies to a given immigration port city.
- it is possible for an airplane to call at multiple US airports, but under the same flight number (see [Can two flights of the same airlines have the same number?](https://www.quora.com/Can-two-flights-of-the-same-airlines-have-the-same-number)).

#### Flight
An airplane on a particular route that arrived at a given immigration port in a given day.

#### Foreign Visitor
A non-US citizen that arrived in a flight for non-immigration purposes, such as pleasure, business or studies.
- it is possible that there actually were other passengers in a flight, US citizens, but we don't have any data about these and are out of scope.

### Star schema

![Star schema](images/de-capstone-star-schema.png)

#### Tables

##### flight_fact table

The only fact table in this model, it contains aggregate measures of a flight's foreign visitors.

The main measure, ```num_visitors```, is the count of each of the resulting foreign visitor categories after grouping by different visitor categories.

Each ```time_id``` and ```route_id``` combination refers to an individual flight, which may have multiple foreign ```visitor_id```s, one for each passenger category combination in that particular flight.

##### foreign_visitor_dim table
Dimension table that represents combinations of foreign visitor categories.

##### time_dim table

Dimension table that represents time with day granularity.

##### route_dim table

Dimension table that represents a route, with many columns that describe the destination.

## ETL

In [ ]:
![Data Flow diagram](images/data_flow.png)

### Data sources

### Extract

The source data is extracted as is into the following staging tables:

- **immigration_staging**(Unnamed: 0, cicid, i94yr, i94mon, i94cit, i94res, i94port, arrdate, i94mode, i94addr, depdate, i94bir, i94visa, count, dtadfile, visapost, occup, entdepa, entdepd, entdepu, matflag, biryear, dtaddto, gender, insnum, airline, admnum, fltno, visatype)


- From immigration data dictionary:
    - **i94port_staging**(port_id, port_desc)
    - **i94country_staging**(country_id, country_desc)
    - **i94mode_staging**(mode_id, mode_desc)
    - **i94state_staging**(state_id, state_desc)
    - **i94visa_staging**(visa_id, visa_desc)


- **airports_staging**(ident, type, name, elevation_ft, continent, iso_country, iso_region, municipality, gps_code, iata_code, local_code, coordinates)


- **demographics_staging**(City, State, Median Age, Male Population, Female Population, Total Population, Number of Veterans, Foreign-born, Average Household Size, State Code, Race, Count)


- **temperatures_staging**(dt, AverageTemperature, AverageTemperatureUncertainty, State, Country)

### Transform

#### Cleaning

The goal is to create views from which the fact and dimension tables can be easily derived, with all data issues solved.

The way I understand cleaning data is quite broad: it consists of normalizing, dealing with missing and out of range values, formatting, eliminating duplicates, enriching, verifying and standardizing. The outcome is rectangular tidy data (columns represent variables and rows represent observations), with tables that have only the required columns with labels suitable for the domain at hand, and that can be easily and correctly joined with other clean tables for further processing.

standardizing: all tables should refer to the same entities by the same set of values
verifying: assert assumptions (like uniqueness)

- **immigration**(arrival_date, airline, flight_number, port, citizenship, residence, age, gender, visa, address_state, stay)
- **port**(port, state, name)
- **state**(state, name, type)
- **country**(country_id, country)
- **airport**(airport_id, name, city, state, coordinates)
- **city**(state, city, size, ethnicity)
- **temperature**(state, month, temperature, climate)

#### Generate association tables:

The goal is to create tables that allow to easily join clean tables, starting from clean tables.

- port_to_city(port_id, state, city)

#### Insert required rows into dimension tables

For each dimension:
1. Figure out which rows are missing, by performing an anti join between each dimension's natural keys and immigration.
2. For each dimension join those missing natural keys with reference tables to fill in all columns.

#### Process fact table

With the immigration table:
1. Join with dimension tables using their natural keys to get corresponding surrogate keys.
2. Group by all dimension natural keys to calculate aggregates.

### Load

Append.

# Scratch area

Do the ETL in a Spark cluster in EMR
- use the cluster's HDFS for storing intermediate results, so that it is efficient to do the different steps as different Airflow tasks.
    - but maybe can just store intermediate results in S3, hopefully data size will be much smaller after the first step, do it this way in the first pass for simplicity
    - remember to clean up at the end of the pipeline
    - let Spark take care of partitioning

### Extract

#### Sources
- immigration monthly SAS
- airports
- state temperatures
- demographics
- i94 data dictionary
- airline.dat

### Transform

##### immigration monthly SAS into flight_fact

want ```flight_fact(arrdate, (airline,fltno,port), citizenship, residence, address, passenger_num)```

    - filter:
        - keep only arrivals by Air
    - drop all except for:
        - arrdate, airline, fltno, i94cit, i94res, i94port, i94addr
    - decode:
        - i94port, i94addr, airline, fltno
    - rename and cast:
        - arrdate as int
        - concat(airline,fltno,port) as string   # in a first pass, later maybe md5 % maxint
        - i94cit as citizenship int
        - i94res as residence int
        - i94addr as address int
        
##### airports

want airports_ref(state, city, coordinates)

    - filter:
        - don't filter by iso_country, this way might keep Guam, etc
    - transform:
        - iso_region into state
        - municipality into city
    - drop all except for:
        - state, city, coordinates
    - group by:
        - state, city and just choose any one coordinate

        
##### state temperatures

want temperatures_ref(state, temperature)

Questions: 
- how to handle missing data (i.e. 2016)?
    - try to find more recent data source or update this one
    - predict with ML
    - just impute latest for same month
- seems that some ports are not in US states, try to join those too

    - filter:
        - as above, keep all countries around just in case
        - keep only latest year
    - rename:
        - AverageTemperature to temperature decimal(3,1)
        - State to state string
    - drop:
        - keep state, temperature

##### demographics

Questions:
- how to handle missing years?
    - in a first pass, treat it as static (with SCD 0 or 1)
- in a first pass, keep just total population

##### i94 data dictionary
    - create country data frame
    
##### airline.dat
    - create airline data frame

#### Dimensions

- country_dim

#### Facts
- from immigration data frame:
    - i94cit, i94res, i94

### Load

- In the cloud
    - Prefer a Data Lake instead of Data Warehouse
    - The reason is that it is yet unclear how frequently this data will be analysed, so provisioning, say, a Redshift cluster could be overkill, no need to pay to have dedicated computation resources and database managed storage.
    - For this reason, blob storage would be preferable
    - This way analysts can use any tool they want (Pandas, Databricks, spark cluster or standalone, Athena, viz tools, etc)
    - If there is high demand, might consider Redshift later on
    - There is no need to stick with Spark, given that data volume is not that high
    
Save everything in **parquet**, because it is structured and distributed

- use fastparquet, pyarrow gives error
    - index=False
    - book recommeds gzip commpression, but start by using default snappy
    - can go into more detail with regards to partitioning for big data case in step 5

- flight_fact:
    - flight_fact.parquet
    - partition_cols=[year, month] #, md5(airline,fltno,port)]
    - in a first pass partition just on (year, month)
- country_dim:
    - country.parquet

## Eating my dog food

Perform some analysis using my schema
- opportunity to show off my pandas and plotting

[Nulls in Fact Tables](https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/kimball-techniques/dimensional-modeling-techniques/fact-table-null/)

> Null-valued measurements behave gracefully in fact tables. The aggregate functions (SUM, COUNT, MIN, MAX, and AVG) all do the “right thing” with null facts. However, nulls must be avoided in the fact table’s foreign keys because these nulls would automatically cause a referential integrity violation. Rather than a null foreign key, the associated dimension table must have a default row (and surrogate key) representing the unknown or not applicable condition.

[Null Attributes in Dimensions](https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/kimball-techniques/dimensional-modeling-techniques/null-dimension-attribute/)
> Null-valued dimension attributes result when a given dimension row has not been fully populated, or when there are attributes that are not applicable to all the dimension’s rows. In both cases, we recommend substituting a descriptive string, such as Unknown or Not Applicable in place of the null value. Nulls in dimension attributes should be avoided because different databases handle grouping and constraining on nulls inconsistently.